In [1]:
from __future__ import annotations

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import yfinance as yf
import time
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")


def _to_dt(s: str) -> datetime:
    return datetime.strptime(s, "%Y-%m-%d")


def _unique_tickers(tickers: list[str]) -> list[str]:
    seen = set()
    out = []
    for t in tickers:
        t = str(t).strip().upper()
        if t and t not in seen:
            out.append(t)
            seen.add(t)
    return out


class StockTracker:
    def __init__(
        self,
        tickers: list[str],
        baseline_date: str,
        verbose: bool = True,
        auto_extend_if_short: bool = True,
        min_trading_days: int = 2,
        extra_backfill_days: int = 10,
    ):
        self.tickers = _unique_tickers(tickers)
        self.baseline_date = baseline_date
        self.verbose = verbose

        # robustness knobs (holiday/weekend edge cases)
        self.auto_extend_if_short = auto_extend_if_short
        self.min_trading_days = int(min_trading_days)
        self.extra_backfill_days = int(extra_backfill_days)

        # one-download store
        self.close: pd.DataFrame = pd.DataFrame()

        # derived
        self.baseline_prices: dict[str, float] = {}
        self.target_prices: dict[str, float] = {}

        # Global window timestamps (based on any ticker availability)
        self.window_start_ts: pd.Timestamp | None = None
        self.window_end_ts: pd.Timestamp | None = None

        # Per-ticker effective baseline date (first valid in window)
        self.per_ticker_baseline_ts: dict[str, pd.Timestamp] = {}

    def _log(self, msg: str):
        logging.info(msg)

    def _download_close(self, start_dt: datetime, end_dt: datetime) -> pd.DataFrame:
        """
        yfinance download helper, returns Close matrix (rows=dates, cols=tickers).
        Handles 1 vs N tickers.
        """
        data = yf.download(
            tickers=self.tickers,
            start=start_dt,
            end=end_dt,
            progress=False,
            auto_adjust=False,
            threads=True,
            group_by="column",
        )

        if data.empty:
            return pd.DataFrame()

        if isinstance(data.columns, pd.MultiIndex):
            close = data["Close"].copy()
        else:
            close = data[["Close"]].copy()
            close.columns = [self.tickers[0]]

        close = close.sort_index()
        return close

    def download_close_once(self, end_date: str) -> pd.DataFrame:
        """
        One download for daily closes:
          baseline_date → end_date (+1 day)
        If too few trading days in the requested window (e.g., holiday/weekend),
        optionally extend backward by extra_backfill_days and re-download.
        """
        if not self.tickers:
            raise ValueError("STOCK_TICKERS is empty.")

        start_dt = _to_dt(self.baseline_date)
        end_dt = _to_dt(end_date) + timedelta(days=1)

        self._log(f"Downloading daily closes ONCE: {self.baseline_date} → {end_date} | tickers={len(self.tickers)}")
        t0 = time.time()

        close = self._download_close(start_dt, end_dt)

        if close.empty:
            self._log("No data returned from yfinance.")
            self.close = pd.DataFrame()
            return self.close

        # determine how many trading rows we have (rows where any ticker has data)
        valid_rows = close.dropna(how="all")
        if self.auto_extend_if_short and len(valid_rows) < self.min_trading_days:
            self._log(
                f"Only {len(valid_rows)} trading day(s) in range. "
                f"Extending backward by {self.extra_backfill_days} days and re-downloading."
            )
            start_dt2 = start_dt - timedelta(days=self.extra_backfill_days)
            close2 = self._download_close(start_dt2, end_dt)
            if not close2.empty:
                close = close2
            else:
                self._log("Extension download returned empty; keeping original window.")

        # forward fill helps missing values for some tickers on some days
        close = close.sort_index().ffill()

        dt_s = time.time() - t0
        self._log(f"Download complete in {dt_s:.2f}s | rows={len(close):,} | cols={len(close.columns)}")
        self.close = close
        return close

    def compute_baseline_and_target(self, end_date: str) -> tuple[dict[str, float], dict[str, float]]:
        """
        Per-ticker baseline/target within the requested window:
          - baseline = first valid close for ticker on/after baseline_date
          - target   = last valid close for ticker on/before end_date
        Also records:
          - global window start/end timestamps (based on any ticker availability)
          - per-ticker effective baseline timestamp
        """
        if self.close.empty:
            return {}, {}

        start_dt = _to_dt(self.baseline_date)
        end_dt = _to_dt(end_date)

        # Filter to the user's requested window (calendar), then drop all-NaN rows
        w = self.close.loc[(self.close.index >= start_dt) & (self.close.index <= end_dt)].copy()
        w = w.dropna(how="all")
        if w.empty:
            self._log("No valid trading rows inside the requested window after filtering.")
            return {}, {}

        self.window_start_ts = w.index[0]
        self.window_end_ts = w.index[-1]

        baseline_prices: dict[str, float] = {}
        target_prices: dict[str, float] = {}
        per_base_ts: dict[str, pd.Timestamp] = {}

        for t in self.tickers:
            if t not in w.columns:
                continue
            s = w[t].dropna()
            if s.empty:
                continue
            per_base_ts[t] = s.index[0]
            baseline_prices[t] = float(s.iloc[0])
            target_prices[t] = float(s.iloc[-1])

        # round for display
        self.baseline_prices = {k: round(v, 2) for k, v in baseline_prices.items()}
        self.target_prices = {k: round(v, 2) for k, v in target_prices.items()}
        self.per_ticker_baseline_ts = per_base_ts

        self._log(f"Window used (global): {self.window_start_ts.date()} → {self.window_end_ts.date()}")
        self._log(f"Valid tickers: {len(self.baseline_prices)}/{len(self.tickers)}")

        if self.verbose:
            self._log(f"Baseline prices: {self.baseline_prices}")
            self._log(f"Target prices:   {self.target_prices}")

        return self.baseline_prices, self.target_prices

    def make_returns_table(self) -> pd.DataFrame:
        """
        Performance table based on per-ticker baseline/target (alignment-safe).
        """
        rows = []
        for t in self.tickers:
            bp = self.baseline_prices.get(t)
            tp = self.target_prices.get(t)
            if bp is None or tp is None or bp == 0:
                continue
            change = tp - bp
            pct = (change / bp) * 100.0
            base_ts = self.per_ticker_baseline_ts.get(t)
            rows.append(
                {
                    "Symbol": t,
                    "Baseline": bp,
                    "Current": tp,
                    "Change": round(change, 2),
                    "Percent_Change": pct,
                    "Baseline_Date_Used": base_ts.date().isoformat() if isinstance(base_ts, pd.Timestamp) else None,
                }
            )

        df = pd.DataFrame(rows)
        if df.empty:
            return df

        df = df.sort_values("Percent_Change", ascending=False).reset_index(drop=True)
        return df

    def make_cumulative_returns_v2(self, end_date: str) -> tuple[pd.DataFrame, pd.Series, pd.DataFrame]:
        """
        Bulletproof v2 cumulative returns:
        - filters close to requested window [baseline_date, end_date]
        - per-ticker baseline alignment (anchor = first valid close per ticker within window)
        - trading-day counter (1..N) for x-axis
        Returns:
          df_cum: wide DataFrame, index=Date, cols=tickers, values=cum return %
          td_counter: Series indexed by Date, value=trading day number
          meta: DataFrame per ticker with effective baseline date + baseline price
        """
        if self.close.empty:
            return pd.DataFrame(), pd.Series(dtype=int), pd.DataFrame()

        start_dt = _to_dt(self.baseline_date)
        end_dt = _to_dt(end_date)

        w = self.close.loc[(self.close.index >= start_dt) & (self.close.index <= end_dt)].copy()
        w = w.dropna(how="all")
        if w.empty:
            return pd.DataFrame(), pd.Series(dtype=int), pd.DataFrame()

        # trading-day counter (only trading rows)
        td_counter = pd.Series(range(1, len(w.index) + 1), index=w.index, name="TradingDay")

        # per-ticker anchor price + baseline date
        anchors = {}
        base_dates = {}
        for t in self.tickers:
            if t not in w.columns:
                continue
            s = w[t].dropna()
            if s.empty:
                continue
            anchors[t] = float(s.iloc[0])
            base_dates[t] = s.index[0]

        if not anchors:
            return pd.DataFrame(), td_counter, pd.DataFrame()

        anchor_series = pd.Series(anchors)

        df_cum = (w.divide(anchor_series, axis="columns") - 1.0) * 100.0
        df_cum = df_cum.dropna(how="all")

        meta = pd.DataFrame(
            {
                "Ticker": list(anchors.keys()),
                "Baseline_Date_Used": [base_dates[t].date().isoformat() for t in anchors.keys()],
                "Baseline_Price_Used": [round(anchors[t], 2) for t in anchors.keys()],
            }
        ).set_index("Ticker")

        return df_cum, td_counter, meta

    def _get_bench_data(self, end_date: str) -> tuple[pd.DataFrame, pd.Series]:
        """Download and normalize benchmark close prices (S&P 500, Nasdaq 100)."""
        bench_symbols = ["^GSPC", "^NDX"]
        bench_label_map = {"^GSPC": "S&P 500", "^NDX": "Nasdaq 100"}
        bench_norm = pd.DataFrame()
        bench_returns = pd.Series(dtype=float)

        bench_raw = yf.download(bench_symbols, start=self.baseline_date, end=end_date)
        if not bench_raw.empty:
            # Reduce to a single price field (prefer Adj Close, else Close)
            if isinstance(bench_raw.columns, pd.MultiIndex):
                level0 = bench_raw.columns.get_level_values(0)
                if "Adj Close" in level0:
                    bench_raw = bench_raw["Adj Close"]
                elif "Close" in level0:
                    bench_raw = bench_raw["Close"]
                else:
                    bench_raw = bench_raw.xs(level0[0], level=0, axis=1)
            else:
                if "Adj Close" in bench_raw.columns:
                    bench_raw = bench_raw[["Adj Close"]]
                elif "Close" in bench_raw.columns:
                    bench_raw = bench_raw[["Close"]]
            bench_raw = bench_raw.ffill().bfill()
            if not bench_raw.empty:
                bench_norm = (bench_raw.divide(bench_raw.iloc[0]) - 1.0) * 100.0
                bench_norm = bench_norm.rename(columns=bench_label_map)
                bench_returns = ((bench_raw.iloc[-1] / bench_raw.iloc[0] - 1.0) * 100.0).rename(index=bench_label_map)

        return bench_norm, bench_returns

    def plot_bar_and_line_v2(self, df_perf: pd.DataFrame, end_date: str, bench_norm=None, bench_returns=None):
        """
        One figure, 2 rows:
          Row 1: total return bar chart
          Row 2: cumulative return paths with trading-day counter x-axis
        Hover shows actual date.
        """
        if df_perf.empty:
            self._log("No performance data to plot.")
            return

        df_cum, td_counter, meta = self.make_cumulative_returns_v2(end_date)

        if bench_norm is None or bench_returns is None:
            bench_norm, bench_returns = self._get_bench_data(end_date)

        if not df_cum.empty and isinstance(bench_norm, pd.DataFrame) and not bench_norm.empty:
            bench_norm = bench_norm.reindex(df_cum.index).ffill()

        bench_dash_cycle = ["dot", "dash"]
        bench_colors = ["#666", "#999"]

        # Titles
        g0 = self.window_start_ts.date().isoformat() if self.window_start_ts is not None else self.baseline_date
        g1 = self.window_end_ts.date().isoformat() if self.window_end_ts is not None else end_date

        title_top = f"📈 Total Return (per-ticker baseline): {g0} → {g1}"
        title_bottom = f"📉 Cumulative Return v2 (Start=0% per ticker) | X = Trading Day Counter"

        fig = make_subplots(
            rows=2,
            cols=1,
            shared_xaxes=False,
            vertical_spacing=0.12,
            subplot_titles=(title_top, title_bottom),
            row_heights=[0.45, 0.55],
        )

        # ----------------
        # Row 1: Bar chart
        # ----------------
        colors = np.where(df_perf["Percent_Change"] >= 0, "#00CC44", "#FF4444")
        fig.add_trace(
            go.Bar(
                x=df_perf["Symbol"],
                y=df_perf["Percent_Change"],
                marker_color=colors,
                text=[f"{x:.1f}%" for x in df_perf["Percent_Change"]],
                textposition="outside",
                hovertemplate=(
                    "<b>%{x}</b><br>"
                    "Baseline: $%{customdata[0]:.2f}<br>"
                    "Current: $%{customdata[1]:.2f}<br>"
                    "Return: %{y:.2f}%<br>"
                    "Baseline Date Used: %{customdata[2]}<extra></extra>"
                ),
                customdata=list(zip(df_perf["Baseline"], df_perf["Current"], df_perf["Baseline_Date_Used"])),
                showlegend=False,
            ),
            row=1,
            col=1,
        )

        avg_change = float(df_perf["Percent_Change"].mean())
        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.4, row=1, col=1)
        fig.add_hline(
            y=avg_change,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Avg: {avg_change:.2f}%",
            annotation_position="bottom right",
            row=1,
            col=1,
        )

        # ----------------------------
        # Row 2: Cum return (TradingDay)
        # ----------------------------
        if not df_cum.empty:
            # Make hover-friendly mapping: Date -> string
            date_str = pd.Series(df_cum.index.strftime("%Y-%m-%d"), index=df_cum.index)

            # Show markers if very short series
            mode = "lines"
            if len(df_cum.index) < 3:
                mode = "lines+markers"

            # plot each ticker that exists
            for t in self.tickers:
                if t not in df_cum.columns:
                    continue
                s = df_cum[t].dropna()
                if s.empty:
                    continue

                x_td = td_counter.loc[s.index].values
                y = s.values
                hover_dates = date_str.loc[s.index].values

                # per-ticker baseline info
                if t in meta.index:
                    bdate = meta.loc[t, "Baseline_Date_Used"]
                    bpx = meta.loc[t, "Baseline_Price_Used"]
                else:
                    bdate, bpx = "N/A", np.nan

                fig.add_trace(
                    go.Scatter(
                        x=x_td,
                        y=y,
                        mode=mode,
                        name=t,
                        customdata=np.column_stack([hover_dates]),
                        hovertemplate=(
                            f"<b>{t}</b><br>"
                            "Trading Day: %{x}<br>"
                            "Date: %{customdata[0]}<br>"
                            "Return: %{y:.2f}%<br>"
                            f"Baseline: {bdate} @ {bpx}<extra></extra>"
                        ),
                    ),
                    row=2,
                    col=1,
                )

            avg_path = df_cum.mean(axis=1, skipna=True)
            if not avg_path.empty:
                fig.add_trace(
                    go.Scatter(
                        x=td_counter.loc[avg_path.index].values,
                        y=avg_path.values,
                        mode="lines",
                        name="Portfolio avg (daily)",
                        line=dict(color="purple", width=3, dash="dashdot"),
                        hovertemplate="Trading Day: %{x}<br>Avg return: %{y:.2f}%<extra></extra>",
                    ),
                    row=2,
                    col=1,
                )

            if not bench_norm.empty:
                for i, col in enumerate(bench_norm.columns):
                    s = bench_norm[col].dropna()
                    if s.empty:
                        continue
                    x_td = td_counter.loc[s.index].values
                    hover_dates = date_str.loc[s.index].values
                    display_name = f"{col} benchmark" if "benchmark" not in str(col).lower() else str(col)
                    fig.add_trace(
                        go.Scatter(
                            x=x_td,
                            y=s.values,
                            mode="lines",
                            name=display_name,
                            line=dict(
                                color=bench_colors[i % len(bench_colors)],
                                dash=bench_dash_cycle[i % len(bench_dash_cycle)],
                                width=2,
                            ),
                            customdata=np.column_stack([hover_dates]),
                            hovertemplate=(
                                f"<b>{display_name}</b><br>"
                                "Trading Day: %{x}<br>"
                                "Date: %{customdata[0]}<br>"
                                "Return: %{y:.2f}%<extra></extra>"
                            ),
                        ),
                        row=2,
                        col=1,
                    )

            fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.4, row=2, col=1)

            # nicer x ticks
            max_td = int(td_counter.max()) if not td_counter.empty else 1
            if max_td <= 15:
                fig.update_xaxes(dtick=1, row=2, col=1)
            elif max_td <= 60:
                fig.update_xaxes(dtick=5, row=2, col=1)
            else:
                fig.update_xaxes(dtick=10, row=2, col=1)

        fig.update_layout(
            height=920,
            width=1120,
            template="plotly_white",
            hovermode="closest",
            title={"text": f"Stock Tracker (One Download, Bulletproof v2) | End: {end_date}", "x": 0.5},
            legend_title_text="Tickers",
        )

        fig.update_xaxes(tickangle=45, row=1, col=1)
        fig.update_yaxes(title_text="Total Return (%)", row=1, col=1)

        fig.update_xaxes(title_text="Trading Day Counter (1 = first trading day in window)", row=2, col=1)
        fig.update_yaxes(title_text="Cumulative Return (%)", row=2, col=1)

        fig.show()


def run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets: dict[str, list[str]] | None = None, verbose: bool = True):
    tracker = StockTracker(
        STOCK_TICKERS,
        BASELINE_DATE,
        verbose=verbose,
        auto_extend_if_short=True,
        min_trading_days=2,
        extra_backfill_days=10,
    )

    tracker.download_close_once(future_date_str)
    tracker.compute_baseline_and_target(future_date_str)

    df_perf = tracker.make_returns_table()
    if df_perf.empty:
        logging.info("No valid tickers with data. Nothing to report.")
        return None

    best = df_perf.iloc[0]
    worst = df_perf.iloc[-1]
    avg = float(df_perf["Percent_Change"].mean())

    bench_norm, bench_returns = tracker._get_bench_data(future_date_str)
    bench_summary = (
        "Bench=" + " | ".join([f"{k}={v:+.2f}%" for k, v in bench_returns.items()])
        if not bench_returns.empty
        else "Bench=N/A"
    )

    batch_summary = ""
    if group_sets:
        group_lookup = {
            "weekly_top5": group_sets.get("weekly", []),
            "pro30_tickers": group_sets.get("pro30", []),
            "movers_tickers": group_sets.get("movers", []),
        }
        batch_chunks = []
        for label, tickers in group_lookup.items():
            if not tickers:
                continue
            tickers_norm = [str(t).strip().upper() for t in tickers if str(t).strip()]
            sub = df_perf[df_perf["Symbol"].isin(tickers_norm)]
            if sub.empty:
                continue
            batch_chunks.append(f"{label}={sub['Percent_Change'].mean():+.2f}%")
        if batch_chunks:
            batch_summary = "Batches=" + " | ".join(batch_chunks)

    summary_msg = (
        f"Summary | Best={best['Symbol']} ({best['Percent_Change']:+.2f}%) "
        f"Worst={worst['Symbol']} ({worst['Percent_Change']:+.2f}%) "
        f"Avg={avg:+.2f}%"
    )

    tail_parts = [x for x in [batch_summary, bench_summary] if x]
    if tail_parts:
        summary_msg += "\n" + " | ".join(tail_parts)

    logging.info(summary_msg)

    tracker.plot_bar_and_line_v2(df_perf, future_date_str, bench_norm=bench_norm, bench_returns=bench_returns)
    return df_perf



In [2]:
import json
from pathlib import Path

def _dedup_keep_order(xs):
    out, seen = [], set()
    for x in xs:
        t = str(x).strip().upper()
        if t and t not in seen:
            out.append(t)
            seen.add(t)
    return out

def load_components_for_date(date_str: str, root: str = "outputs"):
    run_dir = Path(root) / date_str

    weekly = []   # list of dicts: {"ticker":..., "name":...}
    pro30 = []    # list of tickers
    movers = []   # list of tickers

    # Weekly Top 5 (preferred source)
    p_top5 = run_dir / f"weekly_scanner_top5_{date_str}.json"
    if p_top5.exists():
        obj = json.loads(p_top5.read_text())
        for x in obj.get("top5", []):
            if isinstance(x, dict) and x.get("ticker"):
                weekly.append({"ticker": x["ticker"], "name": x.get("name", "")})
    else:
        # fallback: hybrid_analysis embeds weekly_top5
        p_hybrid = run_dir / f"hybrid_analysis_{date_str}.json"
        if p_hybrid.exists():
            h = json.loads(p_hybrid.read_text())
            for x in h.get("weekly_top5", []):
                if isinstance(x, dict) and x.get("ticker"):
                    weekly.append({"ticker": x["ticker"], "name": x.get("name", "")})

    # Pro30 candidates (combined)
    p_pro30 = run_dir / f"30d_momentum_candidates_{date_str}.csv"
    if p_pro30.exists():
        df = pd.read_csv(p_pro30)
        if "Ticker" in df.columns:
            pro30 = df["Ticker"].dropna().tolist()
    else:
        # fallback: breakout/reversal files
        for fn in [f"30d_breakout_candidates_{date_str}.csv", f"30d_reversal_candidates_{date_str}.csv"]:
            p = run_dir / fn
            if p.exists():
                df = pd.read_csv(p)
                if "Ticker" in df.columns:
                    pro30 += df["Ticker"].dropna().tolist()

    # Movers (only stored in hybrid_analysis)
    p_hybrid = run_dir / f"hybrid_analysis_{date_str}.json"
    if p_hybrid.exists():
        h = json.loads(p_hybrid.read_text())
        movers = h.get("movers_tickers", []) or []

    weekly_tickers = _dedup_keep_order([w["ticker"] for w in weekly])
    pro30_tickers = _dedup_keep_order(pro30)
    movers_tickers = _dedup_keep_order(movers)

    # map ticker -> name (only guaranteed for weekly)
    name_map = {w["ticker"].strip().upper(): (w.get("name", "") or "") for w in weekly if w.get("ticker")}

    return {
        "weekly": weekly_tickers,
        "pro30": pro30_tickers,
        "movers": movers_tickers,
        "name_map": name_map,
    }

def format_pretty_tracker_block(date_str: str, root: str = "outputs") -> str:
    comp = load_components_for_date(date_str, root=root)
    weekly, pro30, movers = comp["weekly"], comp["pro30"], comp["movers"]

    lines = []
    lines.append(f"# Universe injected from outputs ({date_str})")
    lines.append(f"# - weekly_top5: {', '.join(weekly) if weekly else '(none)'}")
    lines.append(f"# - pro30_tickers: {', '.join(pro30) if pro30 else '(none)'}")
    lines.append(f"# - movers_tickers: {', '.join(movers) if movers else '(none)'}")
    return "\n".join(lines)


In [3]:
future_date_str = datetime.now().strftime("%Y-%m-%d")

In [4]:
BASELINE_DATE = '2026-01-01' 

# ChatGPT
STOCK_TICKERS = [
    'BMNR',    
    'MSTR',  
    'TSLA',  
    'CRCL',
    'IREN',  
    'SOFI',
    'OSCR',
    'BABA',
    'JD',
    'NVDA',
    'AMD',
    'GOOG',
    'AAPL',
    'UPXI',
]

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)

2026-01-14 07:22:04,498 | INFO | Downloading daily closes ONCE: 2026-01-01 → 2026-01-14 | tickers=14
2026-01-14 07:22:10,490 | INFO | Download complete in 5.99s | rows=8 | cols=14
2026-01-14 07:22:10,492 | INFO | Window used (global): 2026-01-02 → 2026-01-13
2026-01-14 07:22:10,492 | INFO | Valid tickers: 14/14
2026-01-14 07:22:10,493 | INFO | Baseline prices: {'BMNR': 31.19, 'MSTR': 157.16, 'TSLA': 438.07, 'CRCL': 83.47, 'IREN': 42.7, 'SOFI': 27.46, 'OSCR': 14.97, 'BABA': 155.74, 'JD': 29.53, 'NVDA': 188.85, 'AMD': 223.47, 'GOOG': 315.32, 'AAPL': 271.01, 'UPXI': 1.91}
2026-01-14 07:22:10,493 | INFO | Target prices:   {'BMNR': 31.22, 'MSTR': 172.99, 'TSLA': 447.2, 'CRCL': 83.46, 'IREN': 52.99, 'SOFI': 27.14, 'OSCR': 17.38, 'BABA': 167.01, 'JD': 29.88, 'NVDA': 185.81, 'AMD': 220.97, 'GOOG': 336.43, 'AAPL': 261.05, 'UPXI': 2.31}


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed
2026-01-14 07:22:11,631 | INFO | Summary | Best=IREN (+24.10%) Worst=AAPL (-3.68%) Avg=+5.78%
Bench=S&P 500=+1.53% | Nasdaq 100=+2.13%


# Backtesting outputs

In [5]:
BASELINE_DATE = "2025-11-17"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-14 07:22:12,063 | INFO | Downloading daily closes ONCE: 2025-11-17 → 2026-01-14 | tickers=3


# Universe injected from outputs (2025-11-17)
# - weekly_top5: (none)
# - pro30_tickers: TWST, PARR, CDTX
# - movers_tickers: (none)


2026-01-14 07:22:13,524 | INFO | Download complete in 1.46s | rows=39 | cols=3
2026-01-14 07:22:13,526 | INFO | Window used (global): 2025-11-17 → 2026-01-13
2026-01-14 07:22:13,526 | INFO | Valid tickers: 3/3
2026-01-14 07:22:13,527 | INFO | Baseline prices: {'TWST': 26.46, 'PARR': 44.99, 'CDTX': 217.91}
2026-01-14 07:22:13,527 | INFO | Target prices:   {'TWST': 40.22, 'PARR': 37.31, 'CDTX': 221.38}
[*********************100%***********************]  2 of 2 completed
2026-01-14 07:22:14,694 | INFO | Summary | Best=TWST (+52.00%) Worst=PARR (-17.07%) Avg=+12.17%
Batches=pro30_tickers=+12.17% | Bench=S&P 500=+4.37% | Nasdaq 100=+3.80%


In [6]:
BASELINE_DATE = "2025-11-18"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

# Universe injected from outputs (2025-11-18)
# - weekly_top5: (none)
# - pro30_tickers: (none)
# - movers_tickers: (none)


ValueError: STOCK_TICKERS is empty.

In [ ]:
BASELINE_DATE = "2025-11-19"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

In [ ]:
BASELINE_DATE = "2025-11-20"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

In [ ]:
BASELINE_DATE = "2025-11-21"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

In [ ]:
BASELINE_DATE = "2025-11-24"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

In [ ]:
BASELINE_DATE = "2025-11-25"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

In [ ]:
BASELINE_DATE = "2025-11-26"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

In [ ]:
BASELINE_DATE = "2025-11-27"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

In [ ]:
BASELINE_DATE = "2025-11-28"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

In [ ]:
BASELINE_DATE = "2025-12-01"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)